<a href="https://colab.research.google.com/github/Sgwswag/Thesis/blob/main/tennis_markov_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import random

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# (a) Standard state for a game
game_states = ["0-0", "0-15", "15-0", "15-15",
               "30-0", "0-30", "40-0", "30-15",
               "15-30", "0-40", "40-15", "15-40",
               "30-30(DEUCE)", "40-30(A-40)", "30-40(40-A)",
               "HOLD", "BREAK"]
s0game = pd.DataFrame(np.zeros((1, 17)), columns=game_states)
s0game["0-0"] = 1

# (b) Standard state for a tie-break
tiebreak_states = ["0-0", "0-1", "1-0", "1-1",
                   "2-0", "0-2", "3-0", "2-1",
                   "1-2", "0-3", "4-0", "3-1",
                   "2-2", "1-3", "0-4", "5-0",
                   "4-1", "3-2", "2-3", "1-4",
                   "0-5", "5-1", "4-2", "3-3",
                   "2-4", "1-5", "5-2", "4-3", "3-4",
                   "2-5", "5-3", "4-4", "3-5", "5-4",
                   "4-5", "5-5", "6-5", "5-6",
                   "6-6", "SETv1", "SETv2", "6-0",
                   "6-1", "6-2", "6-3", "6-4", "4-6",
                   "3-6", "2-6", "1-6", "0-6", "7-7", "7-6", "6-7"]
s0tb = pd.DataFrame(np.zeros((1, 54)), columns=tiebreak_states)
s0tb["0-0"] = 1

# (c) Standard state for a set
set_states = ["0-0", "0-1", "1-0", "1-1",
              "2-0", "0-2", "3-0", "2-1",
              "1-2", "0-3", "4-0", "3-1",
              "2-2", "1-3", "0-4", "5-0",
              "4-1", "3-2", "2-3", "1-4",
              "0-5", "5-1", "4-2", "3-3",
              "2-4", "1-5", "5-2", "4-3", "3-4",
              "2-5", "5-3", "4-4", "3-5", "5-4",
              "4-5", "5-5", "6-5", "5-6",
              "6-6", "SETv1", "SETv2"]
s0set = pd.DataFrame(np.zeros((1, 41)), columns=set_states)
s0set["0-0"] = 1

# (d) Standard state for a match
match_states = ["0-0", "0-1", "1-0", "1-1", "2-0", "0-2", "2-1", "1-2", "V1", "V2"]
s0match = pd.DataFrame(np.zeros((1, 10)), columns=match_states)
s0match["0-0"] = 1


def plot_markov_chain(transition_matrix, title="Markov Chain"):
    """
    Visualizes a Markov Chain represented by a transition matrix.

    :param transition_matrix: Transition matrix as a pandas DataFrame.
    :param title: Title for the graph.
    """
    G = nx.DiGraph()

    # Adding nodes and edges to the graph

    for from_state, row in transition_matrix.iterrows():
        for to_state, prob in row.iteritems():
            if prob > 0:
                G.add_edge(from_state, to_state, weight=round(prob, 2))

    # Graph layout
    pos = nx.kamada_kawai_layout(G)

    # Drawing nodes and edges
    nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', font_size=8, font_color='black', font_weight='bold', width=[d['weight']*5 for (u,v,d) in G.edges(data=True)])

    # Drawing edge labels
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

    # Displaying the graph
    plt.title(title)
    plt.show()


def MCgame(ppoint_server):
    # Define the probability that the receiver wins a point
    ppoint_ret = 1 - ppoint_server

    # Define the states
    STATES = ["0-0", "0-15", "15-0", "15-15",
              "30-0", "0-30", "40-0", "30-15",
              "15-30", "0-40", "40-15", "15-40",
              "30-30(DEUCE)", "40-30(A-40)", "30-40(40-A)",
              "HOLD", "BREAK"]

    # Create a transition matrix initialized with zeros
    tMat = np.zeros((17, 17))

    # Define a helper function to set probabilities
    def set_prob(from_state, to_state, prob):
        tMat[STATES.index(from_state), STATES.index(to_state)] = prob

    # Set the correct probabilities for server winning the point
    server_wins = [("0-0", "15-0"), ("15-0", "30-0"), ("0-15", "15-15"),
                   ("30-0", "40-0"), ("15-15", "30-15"), ("0-30", "15-30"),
                   ("40-0", "HOLD"), ("30-15", "40-15"), ("40-15", "HOLD"),
                   ("40-30(A-40)", "HOLD"), ("0-40", "15-40"),
                   ("15-40", "30-40(40-A)"), ("30-40(40-A)", "30-30(DEUCE)"),
                   ("15-30", "30-30(DEUCE)"), ("30-30(DEUCE)", "40-30(A-40)")]

    for from_state, to_state in server_wins:
        set_prob(from_state, to_state, ppoint_server)

    # Set the correct probabilities for receiver winning the point
    receiver_wins = [("0-0", "0-15"), ("15-0", "15-15"), ("0-15", "0-30"),
                     ("30-0", "30-15"), ("15-15", "15-30"), ("0-30", "0-40"),
                     ("40-0", "40-15"), ("30-15", "30-30(DEUCE)"),
                     ("40-15", "40-30(A-40)"), ("40-30(A-40)", "30-30(DEUCE)"),
                     ("0-40", "BREAK"), ("15-40", "BREAK"),
                     ("30-40(40-A)", "BREAK"), ("15-30", "15-40"),
                     ("30-30(DEUCE)", "30-40(40-A)")]

    for from_state, to_state in receiver_wins:
        set_prob(from_state, to_state, ppoint_ret)

    # Set stationary states
    set_prob("BREAK", "BREAK", 1)
    set_prob("HOLD", "HOLD", 1)

    # Convert to DataFrame for better readability
    tMat_df = pd.DataFrame(tMat, index=STATES, columns=STATES)

    return tMat_df

def resGAME(ppoint_server, s_game, graph=False):
    """
    Computes outcome probabilities for a service game in tennis and optionally plots the Markov Chain graph.

    :param ppoint_server: Probability that the server wins a point. Between 0 and 1.
    :param s_game: State of the game as a pandas DataFrame. To start from a specific state, set 1 in that state.
    :param graph: Boolean indicating whether to display the Markov Chain graph.
    :return: DataFrame containing stable states (HOLD or BREAK) probabilities.
    """
    # Get the Markov transition matrix
    MC_game1 = MCgame(ppoint_server)

    # Compute the outcome probabilities by raising the matrix to a high power
    resGAME = s_game.dot(np.linalg.matrix_power(MC_game1.values, 10000))

    # Plot the Markov Chain graph if requested
    if graph:
        plot_markov_chain(MC_game1, title="Game model Markov Chain")

    return resGAME

def MCtb(ppoint_srv1, ppoint_srv2):
    # Define the states for a tie-break
    STATES = ["0-0","0-1","1-0","1-1","2-0","0-2","3-0","2-1",
              "1-2","0-3","4-0","3-1",
              "2-2","1-3","0-4","5-0",
              "4-1", "3-2","2-3","1-4",
              "0-5","5-1","4-2","3-3",
              "2-4","1-5","5-2","4-3","3-4",
              "2-5","5-3","4-4","3-5","5-4",
              "4-5", "5-5","6-5","5-6",
              "6-6","6-0",
              "6-1","6-2","6-3","6-4","4-6",
              "3-6","2-6","1-6","0-6","7-7","7-6","6-7","SETv1","SETv2"]

    # Create a transition matrix initialized with zeros
    tMat = np.zeros((54, 54))

    tMat[STATES.index("0-0"), STATES.index("1-0")] = ppoint_srv1
    tMat[STATES.index("3-0"), STATES.index("4-0")] = ppoint_srv1
    tMat[STATES.index("2-1"), STATES.index("3-1")] = ppoint_srv1
    tMat[STATES.index("1-2"), STATES.index("2-2")] = ppoint_srv1
    tMat[STATES.index("0-3"), STATES.index("1-3")] = ppoint_srv1
    tMat[STATES.index("4-0"), STATES.index("5-0")] = ppoint_srv1
    tMat[STATES.index("3-1"), STATES.index("4-1")] = ppoint_srv1
    tMat[STATES.index("2-2"), STATES.index("3-2")] = ppoint_srv1
    tMat[STATES.index("1-3"), STATES.index("2-3")] = ppoint_srv1
    tMat[STATES.index("0-4"), STATES.index("1-4")] = ppoint_srv1
    tMat[STATES.index("6-1"), STATES.index("SETv1")] = ppoint_srv1
    tMat[STATES.index("5-2"), STATES.index("6-2")] = ppoint_srv1
    tMat[STATES.index("4-3"), STATES.index("5-3")] = ppoint_srv1
    tMat[STATES.index("3-4"), STATES.index("4-4")] = ppoint_srv1
    tMat[STATES.index("2-5"), STATES.index("3-5")] = ppoint_srv1
    tMat[STATES.index("1-6"), STATES.index("2-6")] = ppoint_srv1
    tMat[STATES.index("6-2"), STATES.index("SETv1")] = ppoint_srv1
    tMat[STATES.index("5-3"), STATES.index("6-3")] = ppoint_srv1
    tMat[STATES.index("4-4"), STATES.index("5-4")] = ppoint_srv1
    tMat[STATES.index("3-5"), STATES.index("4-5")] = ppoint_srv1
    tMat[STATES.index("2-6"), STATES.index("3-6")] = ppoint_srv1
    tMat[STATES.index("6-5"), STATES.index("SETv1")] = ppoint_srv1
    tMat[STATES.index("5-6"), STATES.index("6-6")] = ppoint_srv1
    tMat[STATES.index("6-6"), STATES.index("7-6")] = ppoint_srv1

    tMat[STATES.index("0-0"), STATES.index("0-1")] = 1 - ppoint_srv1
    tMat[STATES.index("3-0"), STATES.index("3-1")] = 1 - ppoint_srv1
    tMat[STATES.index("2-1"), STATES.index("2-2")] = 1 - ppoint_srv1
    tMat[STATES.index("1-2"), STATES.index("1-3")] = 1 - ppoint_srv1
    tMat[STATES.index("0-3"), STATES.index("0-4")] = 1 - ppoint_srv1
    tMat[STATES.index("4-0"), STATES.index("4-1")] = 1 - ppoint_srv1
    tMat[STATES.index("3-1"), STATES.index("3-2")] = 1 - ppoint_srv1
    tMat[STATES.index("2-2"), STATES.index("2-3")] = 1 - ppoint_srv1
    tMat[STATES.index("1-3"), STATES.index("1-4")] = 1 - ppoint_srv1
    tMat[STATES.index("0-4"), STATES.index("0-5")] = 1 - ppoint_srv1
    tMat[STATES.index("6-1"), STATES.index("6-2")] = 1 - ppoint_srv1
    tMat[STATES.index("5-2"), STATES.index("5-3")] = 1 - ppoint_srv1
    tMat[STATES.index("4-3"), STATES.index("4-4")] = 1 - ppoint_srv1
    tMat[STATES.index("3-4"), STATES.index("3-5")] = 1 - ppoint_srv1
    tMat[STATES.index("2-5"), STATES.index("2-6")] = 1 - ppoint_srv1
    tMat[STATES.index("1-6"), STATES.index("SETv2")] = 1 - ppoint_srv1
    tMat[STATES.index("6-2"), STATES.index("6-3")] = 1 - ppoint_srv1
    tMat[STATES.index("5-3"), STATES.index("5-4")] = 1 - ppoint_srv1
    tMat[STATES.index("4-4"), STATES.index("4-5")] = 1 - ppoint_srv1
    tMat[STATES.index("3-5"), STATES.index("3-6")] = 1 - ppoint_srv1
    tMat[STATES.index("2-6"), STATES.index("SETv2")] = 1 - ppoint_srv1
    tMat[STATES.index("6-5"), STATES.index("6-6")] = 1 - ppoint_srv1
    tMat[STATES.index("5-6"), STATES.index("SETv2")] = 1 - ppoint_srv1
    tMat[STATES.index("6-6"), STATES.index("6-7")] = 1 - ppoint_srv1

    tMat[STATES.index("1-0"), STATES.index("1-1")] = ppoint_srv2
    tMat[STATES.index("0-1"), STATES.index("0-2")] = ppoint_srv2
    tMat[STATES.index("2-0"), STATES.index("2-1")] = ppoint_srv2
    tMat[STATES.index("1-1"), STATES.index("1-2")] = ppoint_srv2
    tMat[STATES.index("0-2"), STATES.index("0-3")] = ppoint_srv2
    tMat[STATES.index("5-0"), STATES.index("5-1")] = ppoint_srv2
    tMat[STATES.index("4-1"), STATES.index("4-2")] = ppoint_srv2
    tMat[STATES.index("3-2"), STATES.index("3-3")] = ppoint_srv2
    tMat[STATES.index("2-3"), STATES.index("2-4")] = ppoint_srv2
    tMat[STATES.index("1-4"), STATES.index("1-5")] = ppoint_srv2
    tMat[STATES.index("0-5"), STATES.index("0-6")] = ppoint_srv2
    tMat[STATES.index("6-0"), STATES.index("6-1")] = ppoint_srv2
    tMat[STATES.index("5-1"), STATES.index("5-2")] = ppoint_srv2
    tMat[STATES.index("4-2"), STATES.index("4-3")] = ppoint_srv2
    tMat[STATES.index("3-3"), STATES.index("3-4")] = ppoint_srv2
    tMat[STATES.index("2-4"), STATES.index("2-5")] = ppoint_srv2
    tMat[STATES.index("1-5"), STATES.index("1-6")] = ppoint_srv2
    tMat[STATES.index("0-6"), STATES.index("SETv2")] = ppoint_srv2
    tMat[STATES.index("6-3"), STATES.index("6-4")] = ppoint_srv2
    tMat[STATES.index("5-4"), STATES.index("5-5")] = ppoint_srv2
    tMat[STATES.index("4-5"), STATES.index("4-6")] = ppoint_srv2
    tMat[STATES.index("3-6"), STATES.index("SETv2")] = ppoint_srv2
    tMat[STATES.index("6-4"), STATES.index("6-5")] = ppoint_srv2
    tMat[STATES.index("5-5"), STATES.index("5-6")] = ppoint_srv2
    tMat[STATES.index("4-6"), STATES.index("SETv2")] = ppoint_srv2
    tMat[STATES.index("6-7"), STATES.index("SETv2")] = ppoint_srv2
    tMat[STATES.index("7-6"), STATES.index("7-7")] = ppoint_srv2
    tMat[STATES.index("7-7"), STATES.index("5-6")] = ppoint_srv2

    tMat[STATES.index("1-0"), STATES.index("2-0")] = 1 - ppoint_srv2
    tMat[STATES.index("0-1"), STATES.index("1-1")] = 1 - ppoint_srv2
    tMat[STATES.index("2-0"), STATES.index("3-0")] = 1 - ppoint_srv2
    tMat[STATES.index("1-1"), STATES.index("2-1")] = 1 - ppoint_srv2
    tMat[STATES.index("0-2"), STATES.index("1-2")] = 1 - ppoint_srv2
    tMat[STATES.index("5-0"), STATES.index("6-0")] = 1 - ppoint_srv2
    tMat[STATES.index("4-1"), STATES.index("5-1")] = 1 - ppoint_srv2
    tMat[STATES.index("3-2"), STATES.index("4-2")] = 1 - ppoint_srv2
    tMat[STATES.index("2-3"), STATES.index("3-3")] = 1 - ppoint_srv2
    tMat[STATES.index("1-4"), STATES.index("2-4")] = 1 - ppoint_srv2
    tMat[STATES.index("0-5"), STATES.index("1-5")] = 1 - ppoint_srv2
    tMat[STATES.index("6-0"), STATES.index("SETv1")] = 1 - ppoint_srv2
    tMat[STATES.index("5-1"), STATES.index("6-1")] = 1 - ppoint_srv2
    tMat[STATES.index("4-2"), STATES.index("5-2")] = 1 - ppoint_srv2
    tMat[STATES.index("3-3"), STATES.index("4-3")] = 1 - ppoint_srv2
    tMat[STATES.index("2-4"), STATES.index("3-4")] = 1 - ppoint_srv2
    tMat[STATES.index("1-5"), STATES.index("2-5")] = 1 - ppoint_srv2
    tMat[STATES.index("0-6"), STATES.index("1-6")] = 1 - ppoint_srv2
    tMat[STATES.index("6-3"), STATES.index("SETv1")] = 1 - ppoint_srv2
    tMat[STATES.index("5-4"), STATES.index("6-4")] = 1 - ppoint_srv2
    tMat[STATES.index("4-5"), STATES.index("5-5")] = 1 - ppoint_srv2
    tMat[STATES.index("3-6"), STATES.index("4-6")] = 1 - ppoint_srv2
    tMat[STATES.index("6-4"), STATES.index("SETv1")] = 1 - ppoint_srv2
    tMat[STATES.index("5-5"), STATES.index("6-5")] = 1 - ppoint_srv2
    tMat[STATES.index("4-6"), STATES.index("5-6")] = 1 - ppoint_srv2
    tMat[STATES.index("6-7"), STATES.index("7-7")] = 1 - ppoint_srv2
    tMat[STATES.index("7-6"), STATES.index("SETv1")] = 1 - ppoint_srv2
    tMat[STATES.index("7-7"), STATES.index("6-5")] = 1 - ppoint_srv2

    # Set stationary states

    tMat[STATES.index("SETv1"), STATES.index("SETv1")] = 1
    tMat[STATES.index("SETv2"), STATES.index("SETv2")] = 1

    # Convert to DataFrame for better readability
    tMat_df = pd.DataFrame(tMat, index=STATES, columns=STATES)

    return tMat_df

def resTIE(ppoint_srv1, ppoint_srv2, s_tb, graph=False):
    """
    Computes outcome probabilities for a tennis tie-break.

    :param ppoint_srv1: Probability that the first player wins a point on his serve.
    :param ppoint_srv2: Probability that the second player wins a point on his serve.
    :param s_tb: Initial state of the tie-break as a pandas DataFrame.
    :param graph: Boolean indicating whether to display the Markov Chain graph.
    :return: DataFrame containing probabilities for stable states (SETv1 or SETv2).
    """
    # Get the Markov transition matrix for a tie-break
    MC_tb = MCtb(ppoint_srv1, ppoint_srv2)

    # Compute the outcome probabilities
    resTIE = s_tb.dot(np.linalg.matrix_power(MC_tb.values, 1000))

    # Plot the Markov Chain graph if requested
    if graph:
        plot_markov_chain(MC_tb, title="Tie-break model")

    return resTIE


def MCset(phold1, phold2, ptie1):
    # Define the states for a set
    STATES = ["0-0","0-1","1-0","1-1",
              "2-0","0-2","3-0","2-1",
              "1-2","0-3","4-0","3-1",
              "2-2","1-3","0-4","5-0",
              "4-1", "3-2","2-3","1-4",
              "0-5","5-1","4-2","3-3",
              "2-4","1-5","5-2","4-3","3-4",
              "2-5","5-3","4-4","3-5","5-4",
              "4-5", "5-5","6-5","5-6",
              "6-6","SETv1","SETv2"]

    # Create a transition matrix initialized with zeros
    tMat = np.zeros((41, 41))

    tMat[STATES.index("0-0"), STATES.index("1-0")] = phold1
    tMat[STATES.index("2-0"), STATES.index("3-0")] = phold1
    tMat[STATES.index("1-1"), STATES.index("2-1")] = phold1
    tMat[STATES.index("0-2"), STATES.index("1-2")] = phold1
    tMat[STATES.index("4-0"), STATES.index("5-0")] = phold1
    tMat[STATES.index("3-1"), STATES.index("4-1")] = phold1
    tMat[STATES.index("2-2"), STATES.index("3-2")] = phold1
    tMat[STATES.index("1-3"), STATES.index("2-3")] = phold1
    tMat[STATES.index("0-4"), STATES.index("1-4")] = phold1
    tMat[STATES.index("5-1"), STATES.index("SETv1")] = phold1
    tMat[STATES.index("4-2"), STATES.index("5-2")] = phold1
    tMat[STATES.index("3-3"), STATES.index("4-3")] = phold1
    tMat[STATES.index("2-4"), STATES.index("3-4")] = phold1
    tMat[STATES.index("1-5"), STATES.index("2-5")] = phold1
    tMat[STATES.index("5-3"), STATES.index("SETv1")] = phold1
    tMat[STATES.index("4-4"), STATES.index("5-4")] = phold1
    tMat[STATES.index("3-5"), STATES.index("4-5")] = phold1
    tMat[STATES.index("5-5"), STATES.index("6-5")] = phold1

    tMat[STATES.index("0-0"), STATES.index("0-1")] = 1 - phold1
    tMat[STATES.index("2-0"), STATES.index("2-1")] = 1 - phold1
    tMat[STATES.index("1-1"), STATES.index("1-2")] = 1 - phold1
    tMat[STATES.index("0-2"), STATES.index("0-3")] = 1 - phold1
    tMat[STATES.index("4-0"), STATES.index("4-1")] = 1 - phold1
    tMat[STATES.index("3-1"), STATES.index("3-2")] = 1 - phold1
    tMat[STATES.index("2-2"), STATES.index("2-3")] = 1 - phold1
    tMat[STATES.index("1-3"), STATES.index("1-4")] = 1 - phold1
    tMat[STATES.index("0-4"), STATES.index("0-5")] = 1 - phold1
    tMat[STATES.index("5-1"), STATES.index("5-2")] = 1 - phold1
    tMat[STATES.index("4-2"), STATES.index("4-3")] = 1 - phold1
    tMat[STATES.index("3-3"), STATES.index("3-4")] = 1 - phold1
    tMat[STATES.index("2-4"), STATES.index("2-5")] = 1 - phold1
    tMat[STATES.index("1-5"), STATES.index("SETv2")] = 1 - phold1
    tMat[STATES.index("5-3"), STATES.index("5-4")] = 1 - phold1
    tMat[STATES.index("4-4"), STATES.index("4-5")] = 1 - phold1
    tMat[STATES.index("3-5"), STATES.index("SETv2")] = 1 - phold1
    tMat[STATES.index("5-5"), STATES.index("5-6")] = 1 - phold1

    tMat[STATES.index("1-0"), STATES.index("1-1")] = phold2
    tMat[STATES.index("0-1"), STATES.index("0-2")] = phold2
    tMat[STATES.index("3-0"), STATES.index("3-1")] = phold2
    tMat[STATES.index("2-1"), STATES.index("2-2")] = phold2
    tMat[STATES.index("1-2"), STATES.index("1-3")] = phold2
    tMat[STATES.index("0-3"), STATES.index("0-4")] = phold2
    tMat[STATES.index("5-0"), STATES.index("5-1")] = phold2
    tMat[STATES.index("4-1"), STATES.index("4-2")] = phold2
    tMat[STATES.index("3-2"), STATES.index("3-3")] = phold2
    tMat[STATES.index("2-3"), STATES.index("2-4")] = phold2
    tMat[STATES.index("1-4"), STATES.index("1-5")] = phold2
    tMat[STATES.index("0-5"), STATES.index("SETv2")] = phold2
    tMat[STATES.index("5-2"), STATES.index("5-3")] = phold2
    tMat[STATES.index("4-3"), STATES.index("4-4")] = phold2
    tMat[STATES.index("3-4"), STATES.index("3-5")] = phold2
    tMat[STATES.index("2-5"), STATES.index("SETv2")] = phold2
    tMat[STATES.index("5-4"), STATES.index("5-5")] = phold2
    tMat[STATES.index("4-5"), STATES.index("SETv2")] = phold2
    tMat[STATES.index("5-6"), STATES.index("SETv2")] = phold2
    tMat[STATES.index("6-5"), STATES.index("6-6")] = phold2

    tMat[STATES.index("1-0"), STATES.index("2-0")] = 1 - phold2
    tMat[STATES.index("0-1"), STATES.index("1-1")] = 1 - phold2
    tMat[STATES.index("3-0"), STATES.index("4-0")] = 1 - phold2
    tMat[STATES.index("2-1"), STATES.index("3-1")] = 1 - phold2
    tMat[STATES.index("1-2"), STATES.index("2-2")] = 1 - phold2
    tMat[STATES.index("0-3"), STATES.index("1-3")] = 1 - phold2
    tMat[STATES.index("5-0"), STATES.index("SETv1")] = 1 - phold2
    tMat[STATES.index("4-1"), STATES.index("5-1")] = 1 - phold2
    tMat[STATES.index("3-2"), STATES.index("4-2")] = 1 - phold2
    tMat[STATES.index("2-3"), STATES.index("3-3")] = 1 - phold2
    tMat[STATES.index("1-4"), STATES.index("2-4")] = 1 - phold2
    tMat[STATES.index("0-5"), STATES.index("1-5")] = 1 - phold2
    tMat[STATES.index("5-2"), STATES.index("SETv1")] = 1 - phold2
    tMat[STATES.index("4-3"), STATES.index("5-3")] = 1 - phold2
    tMat[STATES.index("3-4"), STATES.index("4-4")] = 1 - phold2
    tMat[STATES.index("2-5"), STATES.index("3-5")] = 1 - phold2
    tMat[STATES.index("5-4"), STATES.index("SETv1")] = 1 - phold2
    tMat[STATES.index("4-5"), STATES.index("5-5")] = 1 - phold2
    tMat[STATES.index("5-6"), STATES.index("6-6")] = 1 - phold2
    tMat[STATES.index("6-5"), STATES.index("SETv1")] = 1 - phold2

    # Set stationary states
    tMat[STATES.index("SETv1"), STATES.index("SETv1")] = 1
    tMat[STATES.index("SETv2"), STATES.index("SETv2")] = 1

    # Set tie-break cases
    tMat[STATES.index("6-6"), STATES.index("SETv1")] = ptie1
    tMat[STATES.index("6-6"), STATES.index("SETv2")] = 1 - ptie1

    # Convert to DataFrame for better readability
    tMat_df = pd.DataFrame(tMat, index=STATES, columns=STATES)

    return tMat_df

def resSET(phold1, phold2, ptie1, s_set, graph=False):
    """
    Computes outcome probabilities for a tennis set.

    :param phold1: Probability that the first player wins a game on his serve.
    :param phold2: Probability that the second player wins a game on his serve.
    :param ptie1: Probability that the first player wins a tie-break.
    :param s_set: Initial state of the set as a pandas DataFrame.
    :param graph: Boolean indicating whether to display the Markov Chain graph.
    :return: DataFrame containing probabilities for stable states (SETv1 or SETv2).
    """
    # Get the Markov transition matrix for a set
    MC_set = MCset(phold1, phold2, ptie1)

    # Compute the outcome probabilities
    resSET = s_set.dot(np.linalg.matrix_power(MC_set.values, 100))

    # Plot the Markov Chain graph if requested
    if graph:
        plot_markov_chain(MC_set, title="Set model")

    return resSET

import numpy as np
import pandas as pd

def MCmatch(pset_v1):
    """
    Builds the transition matrix for a tennis match.

    :param pset_v1: Probability that player 1 wins a given set.
    :return: Transition matrix as a pandas DataFrame.
    """
    # Define the probability that player 2 wins a given set
    pset_v2 = 1 - pset_v1

    # Define the states for a match
    STATES = ["0-0", "0-1", "1-0", "1-1", "2-0", "0-2", "2-1", "1-2", "V1", "V2"]

    # Create a transition matrix initialized with zeros
    tMat = np.zeros((10, 10))

    # Define a helper function to set probabilities
    def set_prob(from_state, to_state, prob):
        tMat[STATES.index(from_state), STATES.index(to_state)] = prob

    # Set the probabilities for player 1 winning a set
    set_prob("0-0", "1-0", pset_v1)
    set_prob("1-0", "2-0", pset_v1)
    set_prob("0-1", "1-1", pset_v1)
    set_prob("1-1", "2-1", pset_v1)

    # Set the probabilities for player 2 winning a set
    set_prob("0-0", "0-1", pset_v2)
    set_prob("1-0", "1-1", pset_v2)
    set_prob("0-1", "0-2", pset_v2)
    set_prob("1-1", "1-2", pset_v2)

    # Set stationary states
    set_prob("2-0", "V1", 1)
    set_prob("2-1", "V1", 1)
    set_prob("0-2", "V2", 1)
    set_prob("1-2", "V2", 1)
    set_prob("V1", "V1", 1)
    set_prob("V2", "V2", 1)

    # Convert to DataFrame for better readability
    tMat_df = pd.DataFrame(tMat, index=STATES, columns=STATES)

    return tMat_df

def resMATCH(pset_v1, s_match, graph=False):
    """
    Computes outcome probabilities for a tennis match.

    :param pset_v1: Probability that the first player wins a set when the initial score is 0-0.
    :param s_match: Initial state of the match as a pandas DataFrame.
    :param graph: Boolean indicating whether to display the Markov Chain graph.
    :return: DataFrame containing probabilities for final scores (2-0, 2-1, 1-2, 0-2).
    """
    # Get the Markov transition matrix for a match
    MC_match = MCmatch(pset_v1)

    # Compute the outcome probabilities
    resMATCH = s_match.dot(np.linalg.matrix_power(MC_match.values, 5))  # 5 iterations for stationary states

    # Plot the Markov Chain graph if requested
    if graph:
        plot_markov_chain(MC_match, title="Match model")

    return resMATCH

def predict1(gamescore, phold1, phold2, ptie1, pset_v1, s0match, s0set, s0game, s0tb):
    """
    Computes outcome probabilities for a tennis match from a specific game score in the first set.

    :param gamescore: Current game score in the first set, expressed as a string (e.g. '3-4').
    :param phold1: Probability that player 1 wins a game when serving.
    :param phold2: Probability that player 2 wins a game when serving.
    :param ptie1: Probability that player 1 wins a tie-break.
    :param pset_v1: Probability that player 1 wins a set.
    :param s0match, s0set, s0game, s0tb: Initial states for match, set, game, and tie-break.
    :return: DataFrame containing probabilities for final match scores (V1 or V2).
    """
    # Update the set state with the current game score
    s1set = s0set.copy()
    s1set.loc[0, :] = 0
    s1set.loc[0, gamescore] = 1

    # Update the match state with the probabilities of winning the current set
    s1match = s0match.copy()
    set_probabilities = resSET(phold1, phold2, ptie1, s1set)
    s1match.loc[0, "1-0"] = set_probabilities.loc[0, 39]
    s1match.loc[0, "0-1"] = set_probabilities.loc[0, 40]
    s1match.loc[0, "0-0"] = 0

    # Compute the match probabilities
    resMATCH_probabilities = resMATCH(pset_v1, s1match)

    return resMATCH_probabilities

def predict2(setscore, gamescore, phold1, phold2, ptie1, pset_v1, s0match, s0set, s0game, s0tb):
    """
    Computes outcome probabilities for a tennis match during the second set.

    :param setscore: Current set score, expressed as a string (e.g., '1-0' or '0-1').
    :param gamescore: Current game score in the second set, expressed as a string (e.g., '3-4').
    :param phold1, phold2, ptie1, pset_v1: Various probabilities.
    :param s0match, s0set, s0game, s0tb: Initial states for match, set, game, and tie-break.
    :return: DataFrame containing probabilities for final match scores (V1 or V2).
    """
    # Update the match and set states with the current scores
    s1match = s0match.copy()
    s1match.loc[0, :] = 0
    s1match.loc[0, setscore] = 1

    s1set = s0set.copy()
    s1set.loc[0, :] = 0
    s1set.loc[0, gamescore] = 1

    # Calculate probabilities based on the current set score
    if setscore == "1-0":
        s1match.loc[0, "2-0"] = resSET(phold1, phold2, ptie1, s1set).loc[0, 39]
        s1match.loc[0, "1-1"] = resSET(phold1, phold2, ptie1, s1set).loc[0, 40]
        s1match.loc[0, "1-0"] = 0
    elif setscore == "0-1":
        s1match.loc[0, "0-2"] = resSET(phold1, phold2, ptie1, s1set).loc[0, 40]
        s1match.loc[0, "1-1"] = resSET(phold1, phold2, ptie1, s1set).loc[0, 39]
        s1match.loc[0, "0-1"] = 0

    # Compute the match probabilities
    resMATCH_probabilities = resMATCH(pset_v1, s1match)

    return resMATCH_probabilities

def predict3(gamescore, phold1, phold2, ptie1, pset_v1, s0match, s0set, s0game, s0tb):
    """
    Computes outcome probabilities for a tennis match during the third set.

    :param gamescore: Current game score in the third set, expressed as a string (e.g., '3-4').
    :param phold1: Probability that player 1 wins a game when serving.
    :param phold2: Probability that player 2 wins a game when serving.
    :param ptie1: Probability that player 1 wins a tie-break.
    :param pset_v1: Probability that player 1 wins a set.
    :param s0match, s0set, s0game, s0tb: Initial states for match, set, game, and tie-break.
    :return: DataFrame containing probabilities for final match scores (V1 or V2).
    """
    # Set the match and set states to the current scenario (third set, tied at 1-1)
    s1match = s0match.copy()
    s1match.loc[0, :] = 0
    s1match.loc[0, "1-1"] = 1

    s1set = s0set.copy()
    s1set.loc[0, :] = 0
    s1set.loc[0, gamescore] = 1

    # Calculate probabilities for the third set
    set_probabilities = resSET(phold1, phold2, ptie1, s1set)
    s1match.loc[0, "2-1"] = set_probabilities.loc[0, 39]
    s1match.loc[0, "1-2"] = set_probabilities.loc[0, 40]
    s1match.loc[0, "1-1"] = 0

    # Compute the match probabilities
    resMATCH_probabilities = resMATCH(pset_v1, s1match)

    return resMATCH_probabilities

def determiMM(ppoint_srv1, ppoint_srv2, setscore, gamescore, s0match, s0set, s0game, s0tb):
    """
    Computes outcome probabilities for a tennis match at any given score.

    :param ppoint_srv1: Probability that player 1 wins a point on his serve.
    :param ppoint_srv2: Probability that player 2 wins a point on his serve.
    :param setscore: Current set score, expressed as a string (e.g., '1-0').
    :param gamescore: Current game score, expressed as a string (e.g., '3-4').
    :param s0match, s0set, s0game, s0tb: Initial states for match, set, game, and tie-break.
    :return: DataFrame containing probabilities for final match scores (V1 or V2).
    """
    # Computing constants
    phold1 = resGAME(ppoint_srv1, s0game).loc[0, 15]
    phold2 = resGAME(ppoint_srv2, s0game).loc[0, 15]
    ptie1 = resTIE(ppoint_srv1, ppoint_srv2, s0tb).loc[0, 52]
    pset_v1 = resSET(phold1, phold2, ptie1, s0set).loc[0, 39]

    # Modelization based on the current score
    if setscore == "0-0":
        resTEST = predict1(gamescore, phold1, phold2, ptie1, pset_v1, s0match, s0set, s0game, s0tb)
    elif setscore == "1-1":
        resTEST = predict3(gamescore, phold1, phold2, ptie1, pset_v1, s0match, s0set, s0game, s0tb)
    elif setscore in ["1-0", "0-1"]:
        resTEST = predict2(setscore, gamescore, phold1, phold2, ptie1, pset_v1, s0match, s0set, s0game, s0tb)

    return resTEST

ppoint_server = 0.7
ppoint_srv1 = 0.7
ppoint_srv2 = 0.6

res_game = resGAME(ppoint_server, s_game=s0game, graph=False)

print("Match Setup:")
print(f"Player A Service Point Winning Percentage: {ppoint_server}")
print("Probability of each player winning a game:")
print(res_game)
print("\n")

res_tie = resTIE(ppoint_srv1, ppoint_srv2, s_tb = s0tb, graph = False)

print("Match Setup:")
print(f"Player A Service Point Winning Percentage: {ppoint_srv1}")
print(f"Player B Service Point Winning Percentage: {ppoint_srv2}")
print("Probability of each player winning a tiebreak:")
print(res_tie)
print("\n")

# Compute the proabbility of player 1 holding serve
phold1_res_game = resGAME(ppoint_server, s_game=s0game, graph=False)
phold1 = phold1_res_game.loc[0, 15]

# Compute the probability of player 2 holding serve
phold2_res_game = resGAME(ppoint_server, s_game=s0game, graph=False)
phold2 = phold2_res_game.loc[0, 15]

# Compute the probability of player 1 winning a tie-break
ptie1_res_tie = resTIE(ppoint_srv1, ppoint_srv2, s_tb=s0tb, graph=False)
ptie1 = ptie1_res_tie.loc[0, 52]

# Compute the set outcome probabilities
res_set = resSET(phold1=phold1, phold2=phold2, ptie1=ptie1, s_set=s0set, graph=False)

print("Match Setup:")
print(f"Player A Service Point Winning Percentage: {ppoint_srv1}")
print(f"Player B Service Point Winning Percentage: {ppoint_srv2}")

# Display the result
print("Probability of each player winning a set:")
print(res_set)
print("\n")

# Compute the probability of player 1 winning a set
pset_v1 = resSET(phold1, phold2, ptie1, s0set).loc[0, 39]

print("Match Setup:")
print(f"Player A Service Point Winning Percentage: {ppoint_srv1}")
print(f"Player B Service Point Winning Percentage: {ppoint_srv2}")

res_match = resMATCH(pset_v1, s_match = s0match, graph = False)
print("Probability of each player winning a match:")
print(res_match)
print("\n")

setscore = '1-0'   # Current set score
gamescore = '0-3'  # Current game score

# Printing the match setup
print("Match Setup:")
print(f"Player A Service Point Winning Percentage: {ppoint_srv1}")
print(f"Player B Service Point Winning Percentage: {ppoint_srv2}")
print(f"Current Set Score: {setscore}")
print(f"Current Game Score: {gamescore}")

det_MM = determiMM(ppoint_srv1, ppoint_srv2, setscore, gamescore, s0match=s0match, s0set=s0set, s0game=s0game, s0tb=s0tb)
print("Probability of each player winning a match:")
print(det_MM)
print("\n")


import pandas as pd
import random
import multiprocessing

# Function to read player stats from a CSV file
def read_players_stats(csv_file):
    return pd.read_csv(csv_file)

# Function to read tournament matches from a CSV file
def read_tournament_matches(csv_file):
    return pd.read_csv(csv_file)

# Function to simulate a match between two players
def simulate_match(player1, player2, players_stats):
    ppoint_srv1 = players_stats[player1]["ServicePointsWon"]
    ppoint_srv2 = players_stats[player2]["ServicePointsWon"]
    match_result = determiMM(ppoint_srv1, ppoint_srv2, '0-0', '0-0', s0match, s0set, s0game, s0tb)
    return player1 if random.random() < match_result.loc[0, 8] else player2

# Function to simulate the entire tournament
def simulate_tournament(matches, players_stats):

    while len(matches) > 1:
        next_round_matches = []
        for i in range(0, len(matches), 2):
            match1 = matches[i]
            match2 = matches[i + 1]
            winner1 = simulate_match(match1[0], match1[1], players_stats)
            winner2 = simulate_match(match2[0], match2[1], players_stats)
            next_round_matches.append((winner1, winner2))

        matches = next_round_matches

    final_winner = simulate_match(matches[0][0], matches[0][1], players_stats)

    return final_winner

def main():
    player_stats_file = 'https://raw.githubusercontent.com/apma4903/APMA4903-2023-12-06/main/Dataset.csv'
    players_stats = read_players_stats(player_stats_file)

    simulations = 1000
    all_results = []

    actual_winners = {
        2013: 'Rafael Nadal',
        2014: 'Marin Cilic',
        2015: 'Novak Djokovic',
        2016: 'Stan Wawrinka',
        2017: 'Rafael Nadal',
        2018: 'Novak Djokovic',
        2019: 'Rafael Nadal',
        2020: 'Dominic Thiem',
        2021: 'Daniil Medvedev',
        2022: 'Carlos Alcaraz Garfia'
    }

    for year in range(2013,2023):
        tournament_matches_file = f'https://raw.githubusercontent.com/Sgwswag/Thesis/main/tournament_matches_{year}.csv'  # Each year's tournament file
        tournament_matches = read_tournament_matches(tournament_matches_file)



        year_players_stats = players_stats[players_stats['Year'] == year].set_index('Player').T.to_dict('dict')
        initial_matches = [(row['Player1'], row['Player2']) for _, row in tournament_matches.iterrows()]

        yearly_results = {}

        for _ in range(simulations):
            tournament_winner= simulate_tournament(initial_matches.copy(), year_players_stats)
            yearly_results[tournament_winner] = yearly_results.get(tournament_winner, 0) + 1

        for winner, count in yearly_results.items():
            win_percentage = (count / simulations) * 100
            all_results.append([year, winner, count, win_percentage])



    # Create a DataFrame from the results
    results_df = pd.DataFrame(all_results, columns=['Year', 'Player', 'Wins', 'Win Percentage'])

    for year in range(2013, 2023):
        year_df = results_df[results_df['Year'] == year].copy()
        year_df.drop('Year', axis=1, inplace=True)
        year_df.sort_values(by='Win Percentage', ascending=False, inplace=True)
        year_df.reset_index(drop=True, inplace=True)
        print(f"Year {year} - Actual Winner: {actual_winners[year]}")
        print(f"\n{year_df}\n")

if __name__ == "__main__":
    main()

Match Setup:
Player A Service Point Winning Percentage: 0.7
Probability of each player winning a game:
    0    1    2    3    4    5    6    7    8    9    10   11   12   13   14  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

         15        16  
0  0.900789  0.099211  


Match Setup:
Player A Service Point Winning Percentage: 0.7
Player B Service Point Winning Percentage: 0.6
Probability of each player winning a tiebreak:
    0    1    2    3    4    5    6    7    8    9    10   11   12   13   14  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

    15   16   17   18   19   20   21   22   23   24   25   26   27   28   29  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

    30   31   32   33   34   35   36   37             38   39   40   41   42  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.412584e-133  0.0  0.0  0.0  0.0   

    43   44   45   46   47   48   49   50   51   